In [1]:
import time, sys
import web3 as w3; w3 = w3.Account
from objsize import get_deep_size as get_size
from eth_account.messages import encode_defunct as encode_msg
from hashlib import sha256

In [2]:
# print hash
ph = lambda h: h[:16] + '...' + h[-3:]

# random hash
rh = lambda  : sha256(str(time.time()).encode()).hexdigest()

In [53]:
class Account:
    def __init__(self): 
        self.priv, self.pub = Account.keys()
        self.nonce = 0
        
    @staticmethod
    def keys():
        acc = w3.create()
        return acc.privateKey.hex(), acc.address
    
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        return w3.sign_message(m, self.priv)
    
    def verify(self, tx, sig):
        m = encode_msg(bytes(tx))
        return w3.recover_message(m, signature=sig.signature) == self.pub
    
    def mine(self, block, diff, reward, attempts=1000):
        mblock   = MinebleBlock(block, self.pub, diff, reward, block.size())
        mblock_b = bytes(mblock)
        nonce = 0
        for i in range(attempts):
            candidate   = mblock_b + str(nonce).encode()
            candidate_h = sha256(candidate).hexdigest()
            if candidate_h[:diff] == '0'*diff: return MinedBlock(mblock, nonce)
            nonce += 1

In [54]:
acc1 = Account()
acc2 = Account()

ph(acc1.pub), ph(acc2.pub)

('0xC79B981AC79f1e...bB5', '0x522e8f32A743Ce...a7d')

In [55]:
class TX: 
    def __init__(self, fr, to, value, nonce): 
        self.fr, self.to = fr, to
        self.value       = value
        self.nonce       = nonce
        self.time        = time.time()
    def __str__ (self): return ('from:\t'   + ph(self.fr)+
                                '\nto:\t'   + ph(self.to)+
                                '\nvalue:\t'+ str(self.value)+' ether' 
                                '\nnonce:\t'+ str(self.nonce)+
                                '\ntime:\t' + time.ctime(self.time))
    def __bytes__(self): return (self.fr+
                                 self.to+
                                 str(self.value)+
                                 str(self.nonce)+
                                 str(self.time)).encode()
    
tx1 = TX(acc1.pub, acc2.pub, 3, acc1.nonce)
tx2 = TX(acc2.pub, acc1.pub, 9, acc2.nonce)

print(tx1, '\n')
print(tx2)

from:	0xC79B981AC79f1e...bB5
to:	0x522e8f32A743Ce...a7d
value:	3 ether
nonce:	0
time:	Wed Mar 24 22:46:52 2021 

from:	0x522e8f32A743Ce...a7d
to:	0xC79B981AC79f1e...bB5
value:	9 ether
nonce:	0
time:	Wed Mar 24 22:46:52 2021


In [56]:
sig1 = acc1.sign(tx1)
assert acc1.verify(tx1, sig1)

sig2 = acc2.sign(tx2)
assert acc2.verify(tx2, sig2)

In [57]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])

In [58]:
class Block: 
    def __init__(self, txs, prev_hash, number, time):
        self.txs       = txs
        self.prev_hash = prev_hash
        self.number    = number
        self.time      = time
    def size(self): return get_size(self)
    def __str__(self): return ('time:\t\t'        + time.ctime(self.time)+
                               '\nnumber:\t\t'    + str(self.number)+
                               '\nprev_hash:\t'   + ph(self.prev_hash)+
                               '\n\ntxs:\t\t\n\n' + txs2str(self.txs))
    def __bytes__(self): return (txs2str(self.txs)+
                                 self.prev_hash+
                                 str(self.number)+
                                 str(self.time)).encode()
        
block = Block([tx1], rh(), 2, time.time())
print(block)

time:		Wed Mar 24 22:46:54 2021
number:		2
prev_hash:	25b8c5557691f341...a3a

txs:		

from:	0xC79B981AC79f1e...bB5
to:	0x522e8f32A743Ce...a7d
value:	3 ether
nonce:	0
time:	Wed Mar 24 22:46:52 2021



In [59]:
class MinebleBlock(Block):
    def __init__(self, block, miner, diff, reward, size):
        super().__init__(*block.__dict__.values())
        self.miner  = miner
        self.diff   = diff
        self.reward = reward
        self.size   = size
    def __str__(self): return (super().__str__() + 
                               '\nminer:\t\t'    + self.miner+
                               '\ndiff:\t\t'     + str(self.diff)+
                               '\nreward:\t\t'   + str(self.reward)+' ether'+
                               '\nsize:\t\t'     + str(self.size)+' bytes')
    
mblock = MinebleBlock(block, rh(), 3, 100, block.size())
print(mblock)

time:		Wed Mar 24 22:46:54 2021
number:		2
prev_hash:	25b8c5557691f341...a3a

txs:		

from:	0xC79B981AC79f1e...bB5
to:	0x522e8f32A743Ce...a7d
value:	3 ether
nonce:	0
time:	Wed Mar 24 22:46:52 2021

miner:		25a1ab5dda41795b8ab79310ebbde1db1e610317bb2d46b106cd61049d33d949
diff:		3
reward:		100 ether
size:		791 bytes


In [62]:
class MinedBlock(MinebleBlock):
    def __init__(self, mineble_block, nonce):
        mb    = mineble_block
        block = Block(mb.txs, mb.prev_hash, mb.number, mb.time)
        super().__init__(block, mb.miner, mb.diff, mb.reward, mb.size)
        self.nonce         = nonce
        self.mineble_block = mineble_block
        
    def verify_pow(self):
        candidate   = bytes(self.mineble_block)+str(self.nonce).encode()
        candidate_h = sha256(candidate).hexdigest()
        return candidate_h[:self.mineble_block.diff] == '0'*self.mineble_block.diff
    
mined_block = MinedBlock(mblock, 70)

In [63]:
mined_block = acc1.mine(block,3,100,10000)
assert mined_block.verify_pow()

In [64]:
class Blockchain:
    def __init__(): pass